In [4]:
# 載入HFL_global_model
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from Model import TransformerModel
device = torch.device("mps")
ssl_model_path = "ssl_pretrain.pt"
hfl_model_path = "HFL_global_model.pth"
HFL_Global_Model = TransformerModel(
    feature_dim=14,
    d_model=256,
    nhead=8,
    num_layers=4,
    output_dim=1,
    max_seq_length=100,
    dropout=0.1
).to(device)

SSL_Model = TransformerModel(
    feature_dim=9,
    d_model=256,
    nhead=8,
    num_layers=4,
    output_dim=None,
    max_seq_length=5000,
    dropout=0.1
).to(device)

# 載入HFL模型
HFL_Global_Model.load_state_dict(torch.load(hfl_model_path, map_location=device))

# 載入SSL模型 - 需要從checkpoint中提取model_state_dict
ssl_checkpoint = torch.load(ssl_model_path, map_location=device)
if isinstance(ssl_checkpoint, dict) and 'model_state_dict' in ssl_checkpoint:
    SSL_Model.load_state_dict(ssl_checkpoint['model_state_dict'])
else:
    SSL_Model.load_state_dict(ssl_checkpoint)

print("模型載入成功！")

模型載入成功！


In [ ]:
# 使用 Trainer 載入 Dataset
import glob
import os
from config import load_config
from DataLoader import SequenceCSVDataset
from Trainer import FederatedTrainer

# 載入配置（如果沒有 config.yaml，需要先創建一個簡單的配置對象）
try:
    config = load_config('config.yaml')
except:
    # 如果沒有配置文件，創建一個簡單的配置對象
    class SimpleConfig:
        def __init__(self):
            self.data_path = "./data"  # 請根據實際路徑修改
            self.input_length = 96
            self.output_length = 1
            self.features = ['feature1', 'feature2']  # 請根據實際特徵修改
            self.target = 'Power_Demand'
            self.batch_size = 32
            self.device = device
    
    config = SimpleConfig()

# 載入數據集（以第一個客戶端為例）
csv_pattern = os.path.join(config.data_path, "*.csv")
csv_files = sorted(glob.glob(csv_pattern))

if csv_files:
    # 載入第一個客戶端的數據
    csv_file = csv_files[0]
    csv_name = os.path.splitext(os.path.basename(csv_file))[0]
    
    print(f"載入客戶端數據: {csv_name}")
    
    # 創建數據集對象
    dataset = SequenceCSVDataset(
        csv_path=config.data_path,
        csv_name=csv_name,
        input_len=config.input_length,
        output_len=config.output_length,
        features=config.features,
        target=config.target,
        save_path=config.data_path,
        train_ratio=0.8,
        val_ratio=0.1,
        split_type='time_based',
        fit_scalers=False  # 使用已保存的標準化器
    )
    
    # 使用 Trainer 分割數據集
    trainer = FederatedTrainer(HFL_Global_Model, config, device)
    train_dataset, val_dataset, test_dataset = trainer.split_dataset(dataset)
    
    # 創建數據加載器
    train_loader, val_loader, test_loader = trainer.create_data_loaders(
        train_dataset, val_dataset, test_dataset
    )
    
    print(f"\n✓ 數據載入成功！")
    print(f"  - 訓練集大小: {len(train_dataset)} 樣本")
    print(f"  - 驗證集大小: {len(val_dataset)} 樣本")
    print(f"  - 測試集大小: {len(test_dataset)} 樣本")
    print(f"  - 批次大小: {config.batch_size}")
    
    # 查看一個批次的數據形狀
    for inputs, targets in train_loader:
        print(f"\n數據形狀:")
        print(f"  - 輸入: {inputs.shape}")
        print(f"  - 目標: {targets.shape}")
        break
else:
    print(f"錯誤: 在 {config.data_path} 目錄下沒有找到任何 CSV 文件")

In [ ]:
# 使用 Personalizer 獲取每個客戶端的個性化模型
from Personalizer import initialize_personalized_models, save_personalized_models

print("=" * 70)
print("Per-FedAvg 個性化模型初始化")
print("=" * 70)

# 方法 1: 使用配置文件進行個性化
# 確保 config 中包含必要的參數
if not hasattr(config, 'adaptation_lr'):
    config.adaptation_lr = 0.01  # 個性化適應學習率
if not hasattr(config, 'personalization_steps'):
    config.personalization_steps = 10  # 個性化適應步數
if not hasattr(config, 'model_save_path'):
    config.model_save_path = "./models"  # 模型保存路徑

# 指定全局模型路徑（Per-FedAvg 訓練好的模型）
global_model_path = hfl_model_path  # 使用前面載入的 HFL 模型路徑

try:
    # 初始化所有客戶端的個性化模型
    print(f"\n使用全局模型: {global_model_path}")
    print(f"個性化參數:")
    print(f"  - 適應學習率: {config.adaptation_lr}")
    print(f"  - 適應步數: {config.personalization_steps}")
    print(f"  - 設備: {config.device}")
    
    # 獲取所有客戶端的個性化模型狀態字典
    client_models = initialize_personalized_models(config, global_model_path)
    
    print(f"\n✓ 成功獲取 {len(client_models)} 個客戶端的個性化模型")
    print(f"\n客戶端列表:")
    for idx, client_name in enumerate(client_models.keys(), 1):
        print(f"  {idx}. {client_name}")
    
    # 可選：保存個性化模型到磁盤
    save_dir = "personalized_models"
    save_personalized_models(client_models, save_dir)
    
    # 示範：如何使用特定客戶端的個性化模型
    print(f"\n{'=' * 70}")
    print("使用範例：載入特定客戶端的個性化模型")
    print("=" * 70)
    
    if client_models:
        # 獲取第一個客戶端的名稱
        first_client = list(client_models.keys())[0]
        
        print(f"\n示範：載入客戶端 '{first_client}' 的個性化模型")
        
        # 創建新的模型實例
        personalized_model_client_1 = TransformerModel(
            feature_dim=config.feature_dim if hasattr(config, 'feature_dim') else 14,
            d_model=256,
            nhead=8,
            num_layers=4,
            output_dim=None,  # VFL 場景：不需要輸出層
            max_seq_length=100,
            dropout=0.1
        ).to(device)
        
        # 載入個性化後的權重
        personalized_model_client_1.load_state_dict(client_models[first_client])
        personalized_model_client_1.eval()
        
        print(f"✓ 成功載入客戶端 '{first_client}' 的個性化模型")
        print(f"\n模型已準備好用於 VFL 訓練！")
        
        # 示範：使用個性化模型進行前向傳播（不含輸出投影）
        print(f"\n{'=' * 70}")
        print("VFL 場景示範：使用個性化模型生成嵌入向量")
        print("=" * 70)
        
        # 模擬兩個參與方的數據
        batch_size = 8
        seq_length = 96
        
        # Party A 的數據（客戶端 1）
        x_party_a = torch.randn(batch_size, seq_length, 14).to(device)
        
        with torch.no_grad():
            # 使用個性化模型生成嵌入（不含輸出投影）
            embedding_a = personalized_model_client_1.forward_embedding(x_party_a)
        
        print(f"\n✓ Party A 嵌入向量形狀: {embedding_a.shape}")
        print(f"  - 批次大小: {embedding_a.shape[0]}")
        print(f"  - 嵌入維度: {embedding_a.shape[1]}")
        print(f"\n這個嵌入向量可以發送到 Server 進行融合預測！")
        print(f"原始數據 ({x_party_a.shape}) 保持在本地，保護隱私。")
        
except FileNotFoundError as e:
    print(f"\n❌ 錯誤: {e}")
    print("請確認全局模型路徑是否正確")
except Exception as e:
    print(f"\n❌ 發生錯誤: {e}")
    print("請檢查配置和數據路徑是否正確")

print(f"\n{'=' * 70}")
print("個性化模型初始化完成")
print("=" * 70)

In [ ]:
# ============================================================================
# VFL 場景：載入 Weather (雲端) 和 HFL (本地) 的 Dataset
# ============================================================================

import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle
import numpy as np
from DataLoader import SequenceCSVDataset

print("=" * 80)
print("VFL 數據載入：Weather (雲端) + HFL (本地)")
print("=" * 80)

# === 配置參數 ===
data_path = "./data"
weather_csv_name = "Weather"  # 不含 .csv
hfl_csv_name = "processed/Consumer_01"  # 使用第一個消費者作為示例

# HFL 特徵（本地客戶端）
hfl_features = [
    'AC1', 'AC2', 'AC3', 'AC4', 
    'Dish washer', 'Washing Machine', 'Dryer', 'Water heater',
    'TV', 'Microwave', 'Kettle', 'Lighting', 'Refrigerator', 
    'Consumption_Total'
]

# Weather 特徵（雲端）
weather_features = [
    'TemperatureC', 'DewpointC', 'PressurehPa', 
    'WindSpeedKMH', 'WindSpeedGustKMH', 'Humidity',
    'HourlyPrecipMM', 'dailyrainMM', 'SolarRadiationWatts_m2'
]

# 目標變量
target = ['Power_Demand']  # SequenceCSVDataset 需要列表格式

# 序列參數
seq_length = 96
output_length = 1
batch_size = 32

print(f"\n【數據配置】")
print(f"  - Weather 特徵數: {len(weather_features)}")
print(f"  - HFL 特徵數: {len(hfl_features)}")
print(f"  - 序列長度: {seq_length}")
print(f"  - 輸出長度: {output_length}")
print(f"  - 批次大小: {batch_size}")

# ============================================================================
# 步驟 1: 使用 SequenceCSVDataset 載入 Weather 數據（雲端）
# ============================================================================
print(f"\n【步驟 1】載入 Weather 數據（雲端，使用 SequenceCSVDataset）")

# 檢查 Weather CSV 是否有目標變量
weather_df_check = pd.read_csv(f"{data_path}/{weather_csv_name}.csv")
print(f"  - Weather 原始數據形狀: {weather_df_check.shape}")
print(f"  - Weather 可用欄位: {list(weather_df_check.columns)}")

# Weather 數據使用假目標（因為沒有真實目標）
# 我們將創建一個假的目標欄位，但實際上不會使用它
if 'Power_Demand' not in weather_df_check.columns:
    print(f"  ⚠ Weather 沒有 Power_Demand 欄位，將使用 Weather 特徵創建數據集")
    # 由於 SequenceCSVDataset 需要目標欄位，我們需要手動處理 Weather 數據
    
    # 手動載入和標準化 Weather 數據
    weather_data_raw = weather_df_check[weather_features].values
    
    # Weather 標準化器
    weather_scaler_path = f"{data_path}/weather_scaler.pkl"
    if os.path.exists(weather_scaler_path):
        with open(weather_scaler_path, 'rb') as f:
            weather_scaler = pickle.load(f)
        print(f"  ✓ Weather 標準化器已載入")
    else:
        weather_scaler = StandardScaler()
        weather_scaler.fit(weather_data_raw)
        with open(weather_scaler_path, 'wb') as f:
            pickle.dump(weather_scaler, f)
        print(f"  ✓ Weather 標準化器已創建並保存")
    
    weather_data_scaled = weather_scaler.transform(weather_data_raw)
    print(f"  ✓ Weather 數據已標準化: {weather_data_scaled.shape}")

# ============================================================================
# 步驟 2: 使用 SequenceCSVDataset 載入 HFL 數據（本地）
# ============================================================================
print(f"\n【步驟 2】載入 HFL 數據（本地，使用 SequenceCSVDataset）")

# 檢查 HFL 數據是否有目標
hfl_df_check = pd.read_csv(f"{data_path}/{hfl_csv_name}.csv")
print(f"  - HFL 原始數據形狀: {hfl_df_check.shape}")

# 確定目標變量
if 'Power_Demand' not in hfl_df_check.columns:
    if 'Consumption_Total' in hfl_df_check.columns:
        target = ['Consumption_Total']
        print(f"  ⚠ 使用 'Consumption_Total' 作為目標")
    else:
        raise ValueError("HFL 數據中沒有找到有效的目標變量")

# 使用 SequenceCSVDataset 載入 HFL 數據
try:
    hfl_dataset = SequenceCSVDataset(
        csv_path=data_path,
        csv_name=hfl_csv_name,
        input_len=seq_length,
        output_len=output_length,
        features=hfl_features,
        target=target,
        save_path=data_path,
        train_ratio=0.8,
        val_ratio=0.1,
        split_type='time_based',
        fit_scalers=True  # 第一次運行時創建標準化器
    )
    print(f"  ✓ HFL Dataset 已創建")
    print(f"    - 總序列數: {len(hfl_dataset)}")
    
    # 保存標準化器供後續使用
    hfl_scaler = hfl_dataset.feature_scaler
    target_scaler = hfl_dataset.target_scaler
    
    # 將標準化器保存到統一位置
    with open(f"{data_path}/hfl_scaler.pkl", 'wb') as f:
        pickle.dump(hfl_scaler, f)
    with open(f"{data_path}/target_scaler.pkl", 'wb') as f:
        pickle.dump(target_scaler, f)
    print(f"  ✓ 標準化器已保存到 {data_path}/")
    
except FileNotFoundError as e:
    print(f"  ⚠ 標準化器不存在，使用 fit_scalers=True 創建")
    hfl_dataset = SequenceCSVDataset(
        csv_path=data_path,
        csv_name=hfl_csv_name,
        input_len=seq_length,
        output_len=output_length,
        features=hfl_features,
        target=target,
        save_path=data_path,
        train_ratio=0.8,
        val_ratio=0.1,
        split_type='time_based',
        fit_scalers=True
    )
    hfl_scaler = hfl_dataset.feature_scaler
    target_scaler = hfl_dataset.target_scaler

# ============================================================================
# 步驟 3: 創建匹配的 Weather 序列數據
# ============================================================================
print(f"\n【步驟 3】創建匹配的 Weather 序列數據")

# 創建與 HFL 對應的 Weather 序列
def create_weather_sequences(weather_data, seq_len, total_len):
    """創建 Weather 時序序列"""
    sequences = []
    for i in range(min(len(weather_data) - seq_len + 1, total_len)):
        sequences.append(weather_data[i:i+seq_len])
    return np.array(sequences)

# 獲取 HFL 的總序列數
total_hfl_sequences = len(hfl_dataset)
weather_sequences = create_weather_sequences(weather_data_scaled, seq_length, total_hfl_sequences)

print(f"  ✓ Weather 序列已創建: {weather_sequences.shape}")
print(f"  ✓ HFL 序列數量: {total_hfl_sequences}")

# 確保兩者長度匹配
min_len = min(len(weather_sequences), total_hfl_sequences)
weather_sequences = weather_sequences[:min_len]
print(f"  ✓ 對齊後序列數量: {min_len}")

# ============================================================================
# 步驟 4: 使用 SequenceCSVDataset 的分割索引
# ============================================================================
print(f"\n【步驟 4】分割訓練/驗證/測試集（使用 SequenceCSVDataset 的分割）")

# 獲取 HFL dataset 的分割索引
train_indices = hfl_dataset.train_indices
val_indices = hfl_dataset.val_indices
test_indices = hfl_dataset.test_indices

print(f"  ✓ 訓練集索引: {len(train_indices)} 個序列")
print(f"  ✓ 驗證集索引: {len(val_indices)} 個序列")
print(f"  ✓ 測試集索引: {len(test_indices)} 個序列")

# 分割 Weather 序列
X_weather_train = weather_sequences[train_indices]
X_weather_val = weather_sequences[val_indices]
X_weather_test = weather_sequences[test_indices]

# 從 HFL dataset 獲取數據
train_dataset = hfl_dataset.get_train_dataset()
val_dataset = hfl_dataset.get_val_dataset()
test_dataset = hfl_dataset.get_test_dataset()

# 提取 HFL 序列和目標
X_hfl_train = np.array([hfl_dataset.input_seq[i] for i in train_indices])
X_hfl_val = np.array([hfl_dataset.input_seq[i] for i in val_indices])
X_hfl_test = np.array([hfl_dataset.input_seq[i] for i in test_indices])

y_train = np.array([hfl_dataset.output_seq[i] for i in train_indices])
y_val = np.array([hfl_dataset.output_seq[i] for i in val_indices])
y_test = np.array([hfl_dataset.output_seq[i] for i in test_indices])

print(f"\n  【數據形狀】")
print(f"    訓練集:")
print(f"      - Weather: {X_weather_train.shape}")
print(f"      - HFL: {X_hfl_train.shape}")
print(f"      - Target: {y_train.shape}")
print(f"    驗證集:")
print(f"      - Weather: {X_weather_val.shape}")
print(f"      - HFL: {X_hfl_val.shape}")
print(f"      - Target: {y_val.shape}")
print(f"    測試集:")
print(f"      - Weather: {X_weather_test.shape}")
print(f"      - HFL: {X_hfl_test.shape}")
print(f"      - Target: {y_test.shape}")

# ============================================================================
# 步驟 5: 創建 PyTorch DataLoader
# ============================================================================
print(f"\n【步驟 5】創建 PyTorch DataLoader")

from torch.utils.data import TensorDataset, DataLoader

# 轉換為 PyTorch Tensor
X_weather_train_t = torch.FloatTensor(X_weather_train).to(device)
X_hfl_train_t = torch.FloatTensor(X_hfl_train).to(device)
y_train_t = torch.FloatTensor(y_train).unsqueeze(1).to(device) if y_train.ndim == 1 else torch.FloatTensor(y_train).to(device)

X_weather_val_t = torch.FloatTensor(X_weather_val).to(device)
X_hfl_val_t = torch.FloatTensor(X_hfl_val).to(device)
y_val_t = torch.FloatTensor(y_val).unsqueeze(1).to(device) if y_val.ndim == 1 else torch.FloatTensor(y_val).to(device)

X_weather_test_t = torch.FloatTensor(X_weather_test).to(device)
X_hfl_test_t = torch.FloatTensor(X_hfl_test).to(device)
y_test_t = torch.FloatTensor(y_test).unsqueeze(1).to(device) if y_test.ndim == 1 else torch.FloatTensor(y_test).to(device)

# 創建 Dataset
train_dataset_vfl = TensorDataset(X_weather_train_t, X_hfl_train_t, y_train_t)
val_dataset_vfl = TensorDataset(X_weather_val_t, X_hfl_val_t, y_val_t)
test_dataset_vfl = TensorDataset(X_weather_test_t, X_hfl_test_t, y_test_t)

# 創建 DataLoader
train_loader_vfl = DataLoader(train_dataset_vfl, batch_size=batch_size, shuffle=True)
val_loader_vfl = DataLoader(val_dataset_vfl, batch_size=batch_size, shuffle=False)
test_loader_vfl = DataLoader(test_dataset_vfl, batch_size=batch_size, shuffle=False)

print(f"  ✓ 訓練 DataLoader: {len(train_loader_vfl)} 批次")
print(f"  ✓ 驗證 DataLoader: {len(val_loader_vfl)} 批次")
print(f"  ✓ 測試 DataLoader: {len(test_loader_vfl)} 批次")

# 保存 create_sequences 函數供後續使用
def create_sequences(weather, hfl, targets, seq_len):
    """創建時序序列數據（供其他客戶端使用）"""
    X_weather, X_hfl, y = [], [], []
    
    for i in range(len(weather) - seq_len):
        X_weather.append(weather[i:i+seq_len])
        X_hfl.append(hfl[i:i+seq_len])
        y.append(targets[i+seq_len])
    
    return np.array(X_weather), np.array(X_hfl), np.array(y)

# ============================================================================
# 數據載入完成
# ============================================================================
print(f"\n{'=' * 80}")
print("✓ VFL 數據載入完成（使用 SequenceCSVDataset）！")
print(f"{'=' * 80}")
print(f"\n數據摘要:")
print(f"  【雲端 - Weather（無目標）】")
print(f"    - 特徵維度: {len(weather_features)}")
print(f"    - 訓練樣本: {X_weather_train.shape}")
print(f"    - 標準化器: {weather_scaler_path}")
print(f"  【本地 - HFL（有目標）】")
print(f"    - 特徵維度: {len(hfl_features)}")
print(f"    - 訓練樣本: {X_hfl_train.shape}")
print(f"    - 使用 SequenceCSVDataset 管理")
print(f"  【目標變量】")
print(f"    - 變量: {target}")
print(f"    - 訓練樣本: {y_train.shape}")
print(f"    - 標準化器: target_scaler (來自 SequenceCSVDataset)")
print(f"\n下一步：建立 Weather Model（雲端）和 Fusion Model（本地）")

In [ ]:
# ============================================================================
# VFL 聯邦學習訓練：多客戶端 + FedAvg + 分階段訓練
# ============================================================================

from Model import TransformerModel, FusionModel
from tqdm import tqdm
import random
import copy

print("=" * 80)
print("VFL 聯邦學習訓練：多客戶端 FedAvg")
print("=" * 80)

# ============================================================================
# 步驟 1: 載入所有客戶端數據
# ============================================================================
print(f"\n【步驟 1】載入所有客戶端數據")

# 獲取所有客戶端的 CSV 文件
import glob
client_csv_files = sorted(glob.glob(f"{data_path}/processed/Consumer_*.csv"))
print(f"  ✓ 找到 {len(client_csv_files)} 個客戶端")

# 為每個客戶端創建數據加載器
client_dataloaders = {}
client_names = []

for csv_file in client_csv_files:
    client_name = os.path.basename(csv_file).replace('.csv', '')
    client_names.append(client_name)
    
    # 載入客戶端數據
    client_df = pd.read_csv(csv_file)
    
    # 檢查目標變量
    if target not in client_df.columns:
        client_target = 'Consumption_Total'
    else:
        client_target = target
    
    # 提取特徵和目標
    client_hfl_data = client_df[hfl_features].values
    client_target_data = client_df[client_target].values
    
    # 標準化
    client_hfl_scaled = hfl_scaler.transform(client_hfl_data)
    client_target_scaled = target_scaler.transform(client_target_data.reshape(-1, 1)).flatten()
    
    # 對齊長度
    min_len = min(len(weather_data_scaled), len(client_hfl_scaled), len(client_target_scaled))
    client_weather = weather_data_scaled[:min_len]
    client_hfl = client_hfl_scaled[:min_len]
    client_targets = client_target_scaled[:min_len]
    
    # 創建序列
    X_w, X_h, y = create_sequences(client_weather, client_hfl, client_targets, seq_length)
    
    # 分割數據集（8:1:1）
    total = len(X_w)
    train_size = int(0.8 * total)
    val_size = int(0.1 * total)
    
    X_w_train = torch.FloatTensor(X_w[:train_size]).to(device)
    X_h_train = torch.FloatTensor(X_h[:train_size]).to(device)
    y_train = torch.FloatTensor(y[:train_size]).unsqueeze(1).to(device)
    
    X_w_val = torch.FloatTensor(X_w[train_size:train_size+val_size]).to(device)
    X_h_val = torch.FloatTensor(X_h[train_size:train_size+val_size]).to(device)
    y_val = torch.FloatTensor(y[train_size:train_size+val_size]).unsqueeze(1).to(device)
    
    # 創建 DataLoader
    train_dataset = TensorDataset(X_w_train, X_h_train, y_train)
    val_dataset = TensorDataset(X_w_val, X_h_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    client_dataloaders[client_name] = {
        'train': train_loader,
        'val': val_loader,
        'train_size': len(train_dataset)
    }
    
    print(f"  ✓ {client_name}: {len(train_dataset)} 訓練樣本, {len(val_dataset)} 驗證樣本")

print(f"\n總共載入 {len(client_dataloaders)} 個客戶端")

# ============================================================================
# 步驟 2: 初始化模型
# ============================================================================
print(f"\n【步驟 2】初始化模型")

# === 雲端 Weather Model（全局，可訓練）===
global_weather_model = TransformerModel(
    feature_dim=len(weather_features),
    d_model=256,
    nhead=8,
    num_layers=4,
    output_dim=None,
    max_seq_length=seq_length,
    dropout=0.1
).to(device)

print(f"  ✓ 全局 Weather Model 已初始化（雲端）")
print(f"    - 參數: {sum(p.numel() for p in global_weather_model.parameters()):,}")

# === 為每個客戶端創建本地模型 ===
client_models = {}

for client_name in client_names:
    # HFL Model（本地，凍結）
    hfl_model = TransformerModel(
        feature_dim=len(hfl_features),
        d_model=256,
        nhead=8,
        num_layers=4,
        output_dim=None,
        max_seq_length=seq_length,
        dropout=0.1
    ).to(device)
    
    # 嘗試載入個性化權重
    if 'client_models' in locals() and client_name.replace('Consumer_', '') in str(client_models):
        # 載入個性化權重
        pass  # 可以從前面的 Personalizer 獲得
    else:
        # 使用隨機初始化或載入全局 HFL 模型
        pass
    
    # 凍結 HFL Model
    for param in hfl_model.parameters():
        param.requires_grad = False
    hfl_model.eval()
    
    # Fusion Model（本地，可訓練）
    fusion_model = FusionModel(
        embedding_dim_party_a=256,
        embedding_dim_party_b=256,
        hidden_dim=256,
        output_dim=1,
        dropout=0.1
    ).to(device)
    
    client_models[client_name] = {
        'hfl_model': hfl_model,
        'fusion_model': fusion_model
    }

print(f"  ✓ 為 {len(client_models)} 個客戶端創建本地模型")

# ============================================================================
# 步驟 3: 訓練配置
# ============================================================================
print(f"\n【步驟 3】訓練配置")

# 訓練參數
num_clients = 5  # 每輪選擇的客戶端數量
total_rounds = 100  # 總訓練輪數
phase1_rounds = 10  # 前 10 輪：每個 epoch 都訓練兩個模型
phase2_rounds = 90  # 後 90 輪：4 個 epoch 只訓練 Fusion，1 個 epoch 訓練雲端

learning_rate = 0.001
criterion = nn.MSELoss()

print(f"  【聯邦學習配置】")
print(f"    - 總訓練輪數: {total_rounds}")
print(f"    - 每輪選擇客戶端數: {num_clients} / {len(client_names)}")
print(f"    - 階段 1 (輪 1-{phase1_rounds}): 每個 epoch 都訓練 Fusion + Weather")
print(f"    - 階段 2 (輪 {phase1_rounds+1}-{total_rounds}): 4 epoch 訓練 Fusion，1 epoch 訓練 Weather")
print(f"    - 學習率: {learning_rate}")

# ============================================================================
# 步驟 4: FedAvg 梯度聚合函數
# ============================================================================

def aggregate_gradients(client_gradients, client_weights):
    """
    FedAvg 梯度聚合
    
    Args:
        client_gradients: List of gradients from each client
        client_weights: List of weights (based on dataset size)
    
    Returns:
        Aggregated gradients
    """
    aggregated_grads = []
    
    # 正規化權重
    total_weight = sum(client_weights)
    normalized_weights = [w / total_weight for w in client_weights]
    
    # 對每個參數進行加權平均
    for param_idx in range(len(client_gradients[0])):
        weighted_grad = None
        for client_idx, grads in enumerate(client_gradients):
            if weighted_grad is None:
                weighted_grad = grads[param_idx] * normalized_weights[client_idx]
            else:
                weighted_grad += grads[param_idx] * normalized_weights[client_idx]
        aggregated_grads.append(weighted_grad)
    
    return aggregated_grads

def apply_aggregated_gradients(model, aggregated_grads):
    """將聚合後的梯度應用到模型"""
    for param, grad in zip(model.parameters(), aggregated_grads):
        if param.grad is None:
            param.grad = grad.clone()
        else:
            param.grad.copy_(grad)

# ============================================================================
# 步驟 5: 訓練循環
# ============================================================================
print(f"\n{'=' * 80}")
print("開始聯邦學習訓練...")
print("=" * 80)

# 記錄訓練歷史
history = {
    'train_loss': [],
    'val_loss': [],
    'selected_clients': []
}

# 全局優化器
global_weather_optimizer = torch.optim.Adam(
    global_weather_model.parameters(),
    lr=learning_rate,
    weight_decay=1e-4
)

for round_idx in range(total_rounds):
    print(f"\n{'─' * 80}")
    print(f"聯邦學習輪次 [{round_idx + 1}/{total_rounds}]")
    print(f"{'─' * 80}")
    
    # 確定訓練策略
    if round_idx < phase1_rounds:
        # 階段 1: 每個 epoch 都訓練兩個模型
        train_weather = True
        phase_name = "階段 1"
    else:
        # 階段 2: 4 epoch 訓練 Fusion，1 epoch 訓練 Weather
        epoch_in_phase2 = (round_idx - phase1_rounds) % 5
        train_weather = (epoch_in_phase2 == 4)  # 每 5 個 epoch 的第 5 個訓練 Weather
        phase_name = "階段 2"
    
    print(f"  【{phase_name}】訓練模式: ", end="")
    if train_weather:
        print("Fusion Model + Weather Model ⚡")
    else:
        print("Fusion Model only (節省通訊) 📡")
    
    # ========================================================================
    # 客戶端選擇
    # ========================================================================
    available_clients = list(client_names)
    selected_clients = random.sample(available_clients, min(num_clients, len(available_clients)))
    
    print(f"\n  【客戶端選擇】")
    print(f"    選中: {selected_clients}")
    
    history['selected_clients'].append(selected_clients)
    
    # ========================================================================
    # 客戶端本地訓練
    # ========================================================================
    client_losses = []
    client_weather_gradients = []
    client_sample_counts = []
    
    for client_name in selected_clients:
        # 獲取客戶端模型和數據
        hfl_model = client_models[client_name]['hfl_model']
        fusion_model = client_models[client_name]['fusion_model']
        train_loader = client_dataloaders[client_name]['train']
        
        # 創建本地 Weather Model 副本（用於計算梯度）
        local_weather_model = copy.deepcopy(global_weather_model)
        local_weather_model.train()
        
        # 本地優化器
        fusion_optimizer = torch.optim.Adam(
            fusion_model.parameters(),
            lr=learning_rate,
            weight_decay=1e-4
        )
        
        if train_weather:
            weather_optimizer = torch.optim.Adam(
                local_weather_model.parameters(),
                lr=learning_rate,
                weight_decay=1e-4
            )
        
        # 本地訓練
        fusion_model.train()
        epoch_loss = 0.0
        num_batches = 0
        
        for weather_batch, hfl_batch, targets in train_loader:
            # 前向傳播
            if train_weather:
                # 訓練 Weather Model
                weather_embedding = local_weather_model.forward_embedding(weather_batch)
            else:
                # 不訓練 Weather Model，使用全局模型
                with torch.no_grad():
                    weather_embedding = global_weather_model.forward_embedding(weather_batch)
            
            with torch.no_grad():
                hfl_embedding = hfl_model.forward_embedding(hfl_batch)
            
            # Fusion 預測
            predictions = fusion_model(weather_embedding, hfl_embedding)
            loss = criterion(predictions, targets)
            
            # 反向傳播
            fusion_optimizer.zero_grad()
            if train_weather:
                weather_optimizer.zero_grad()
            
            loss.backward()
            
            # 更新 Fusion Model
            fusion_optimizer.step()
            
            # 更新 Weather Model
            if train_weather:
                weather_optimizer.step()
            
            epoch_loss += loss.item()
            num_batches += 1
        
        avg_loss = epoch_loss / num_batches
        client_losses.append(avg_loss)
        
        # 收集 Weather Model 梯度（用於 FedAvg）
        if train_weather:
            grads = [param.grad.clone() if param.grad is not None else torch.zeros_like(param) 
                     for param in local_weather_model.parameters()]
            client_weather_gradients.append(grads)
            client_sample_counts.append(client_dataloaders[client_name]['train_size'])
        
        print(f"    ✓ {client_name}: Loss = {avg_loss:.6f}")
    
    # ========================================================================
    # 雲端聚合（FedAvg）
    # ========================================================================
    if train_weather and len(client_weather_gradients) > 0:
        print(f"\n  【雲端聚合】FedAvg 梯度融合")
        
        # 聚合梯度
        aggregated_grads = aggregate_gradients(client_weather_gradients, client_sample_counts)
        
        # 應用聚合梯度到全局模型
        apply_aggregated_gradients(global_weather_model, aggregated_grads)
        global_weather_optimizer.step()
        
        print(f"    ✓ 全局 Weather Model 已更新")
        print(f"    - 參與客戶端: {len(client_weather_gradients)}")
        print(f"    - 樣本權重: {[f'{w/sum(client_sample_counts):.2%}' for w in client_sample_counts]}")
    
    # ========================================================================
    # 全局驗證
    # ========================================================================
    global_weather_model.eval()
    val_losses = []
    
    for client_name in selected_clients:
        fusion_model = client_models[client_name]['fusion_model']
        hfl_model = client_models[client_name]['hfl_model']
        val_loader = client_dataloaders[client_name]['val']
        
        fusion_model.eval()
        val_loss = 0.0
        num_batches = 0
        
        with torch.no_grad():
            for weather_batch, hfl_batch, targets in val_loader:
                weather_embedding = global_weather_model.forward_embedding(weather_batch)
                hfl_embedding = hfl_model.forward_embedding(hfl_batch)
                predictions = fusion_model(weather_embedding, hfl_embedding)
                loss = criterion(predictions, targets)
                val_loss += loss.item()
                num_batches += 1
        
        val_losses.append(val_loss / num_batches if num_batches > 0 else 0)
    
    # 記錄歷史
    avg_train_loss = sum(client_losses) / len(client_losses)
    avg_val_loss = sum(val_losses) / len(val_losses)
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(avg_val_loss)
    
    print(f"\n  【本輪結果】")
    print(f"    平均訓練損失: {avg_train_loss:.6f}")
    print(f"    平均驗證損失: {avg_val_loss:.6f}")

print(f"\n{'=' * 80}")
print("✓ 聯邦學習訓練完成！")
print("=" * 80)

# ============================================================================
# 步驟 6: 評估與可視化
# ============================================================================
print(f"\n【步驟 6】全局評估")

# 在所有客戶端上評估
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

all_predictions = []
all_targets = []

global_weather_model.eval()

for client_name in client_names[:5]:  # 評估前 5 個客戶端
    fusion_model = client_models[client_name]['fusion_model']
    hfl_model = client_models[client_name]['hfl_model']
    val_loader = client_dataloaders[client_name]['val']
    
    fusion_model.eval()
    
    with torch.no_grad():
        for weather_batch, hfl_batch, targets in val_loader:
            weather_embedding = global_weather_model.forward_embedding(weather_batch)
            hfl_embedding = hfl_model.forward_embedding(hfl_batch)
            predictions = fusion_model(weather_embedding, hfl_embedding)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

all_predictions = np.array(all_predictions)
all_targets = np.array(all_targets)

mse = mean_squared_error(all_targets, all_predictions)
mae = mean_absolute_error(all_targets, all_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(all_targets, all_predictions)

print(f"\n全局性能（前 5 個客戶端）:")
print(f"  - MSE: {mse:.6f}")
print(f"  - MAE: {mae:.6f}")
print(f"  - RMSE: {rmse:.6f}")
print(f"  - R²: {r2:.6f}")

# ============================================================================
# 可視化
# ============================================================================
print(f"\n【步驟 7】可視化結果")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. 訓練損失曲線
axes[0, 0].plot(history['train_loss'], label='Train Loss', color='blue', linewidth=2)
axes[0, 0].plot(history['val_loss'], label='Val Loss', color='orange', linewidth=2)
axes[0, 0].axvline(x=phase1_rounds, color='red', linestyle='--', label='Phase 1→2', linewidth=2)
axes[0, 0].set_xlabel('Round')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Federated Learning: Training Progress')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 預測 vs 實際
axes[0, 1].scatter(all_targets, all_predictions, alpha=0.5, s=10)
axes[0, 1].plot([all_targets.min(), all_targets.max()], 
                [all_targets.min(), all_targets.max()], 
                'r--', lw=2, label='Perfect Prediction')
axes[0, 1].set_xlabel('Actual')
axes[0, 1].set_ylabel('Predicted')
axes[0, 1].set_title(f'Predictions vs Actual (R²={r2:.4f})')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 客戶端參與分佈
from collections import Counter
client_participation = Counter([c for round_clients in history['selected_clients'] for c in round_clients])
axes[1, 0].bar(range(len(client_participation)), list(client_participation.values()), color='skyblue')
axes[1, 0].set_xlabel('Client Index')
axes[1, 0].set_ylabel('Participation Count')
axes[1, 0].set_title('Client Participation Distribution')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. 誤差分佈
errors = all_predictions.flatten() - all_targets.flatten()
axes[1, 1].hist(errors, bins=50, alpha=0.7, color='green', edgecolor='black')
axes[1, 1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[1, 1].set_xlabel('Prediction Error')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title(f'Error Distribution (MAE={mae:.4f})')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('federated_vfl_results.png', dpi=150, bbox_inches='tight')
print(f"  ✓ 可視化圖表已保存: federated_vfl_results.png")
plt.show()

print(f"\n{'=' * 80}")
print("VFL 聯邦學習完成！")
print("=" * 80)
print(f"\n最終統計:")
print(f"  - 總訓練輪數: {total_rounds}")
print(f"  - 參與客戶端總數: {len(client_names)}")
print(f"  - 每輪選擇: {num_clients} 個客戶端")
print(f"  - Weather Model 更新次數: {phase1_rounds + phase2_rounds // 5}")
print(f"  - 通訊節省: {(1 - (phase1_rounds + phase2_rounds // 5) / total_rounds) * 100:.1f}%")
print(f"\n模型架構:")
print(f"  ✓ Weather Model: 雲端（共享）")
print(f"  ✓ HFL Models: 本地（每個客戶端，凍結）")
print(f"  ✓ Fusion Models: 本地（每個客戶端，可訓練）")

In [ ]:
# ============================================================================
# 反標準化與詳細可視化
# ============================================================================

print("=" * 80)
print("反標準化與詳細可視化")
print("=" * 80)

# ============================================================================
# 步驟 1: 反標準化預測結果
# ============================================================================
print(f"\n【步驟 1】反標準化預測結果")

# 反標準化預測值和實際值
predictions_original = target_scaler.inverse_transform(all_predictions.reshape(-1, 1)).flatten()
targets_original = target_scaler.inverse_transform(all_targets.reshape(-1, 1)).flatten()

print(f"  ✓ 已完成反標準化")
print(f"    - 預測值範圍: [{predictions_original.min():.2f}, {predictions_original.max():.2f}]")
print(f"    - 實際值範圍: [{targets_original.min():.2f}, {targets_original.max():.2f}]")

# 重新計算評估指標（使用原始尺度）
mse_original = mean_squared_error(targets_original, predictions_original)
mae_original = mean_absolute_error(targets_original, predictions_original)
rmse_original = np.sqrt(mse_original)
r2_original = r2_score(targets_original, predictions_original)

# 計算 sMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0  # 避免除零
    return np.mean(diff) * 100

smape_value = smape(targets_original, predictions_original)

print(f"\n【原始尺度評估指標】")
print(f"  - MSE: {mse_original:.4f}")
print(f"  - MAE: {mae_original:.4f}")
print(f"  - RMSE: {rmse_original:.4f}")
print(f"  - R²: {r2_original:.4f}")
print(f"  - sMAPE: {smape_value:.2f}%")

# ============================================================================
# 步驟 2: 詳細可視化
# ============================================================================
print(f"\n【步驟 2】生成詳細可視化")

fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# === 圖 1: 訓練損失曲線 ===
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(history['train_loss'], label='Train Loss', color='blue', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', color='orange', linewidth=2)
ax1.axvline(x=phase1_rounds, color='red', linestyle='--', label='Phase 1→2', linewidth=2)
ax1.set_xlabel('Round', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Federated Learning: Training Progress', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# === 圖 2: 預測 vs 實際（散點圖，原始尺度）===
ax2 = fig.add_subplot(gs[0, 1])
ax2.scatter(targets_original, predictions_original, alpha=0.5, s=20, c='steelblue', edgecolors='black', linewidth=0.5)
ax2.plot([targets_original.min(), targets_original.max()], 
         [targets_original.min(), targets_original.max()], 
         'r--', lw=3, label='Perfect Prediction')
ax2.set_xlabel('Actual Power Demand (原始值)', fontsize=12)
ax2.set_ylabel('Predicted Power Demand (原始值)', fontsize=12)
ax2.set_title(f'Predictions vs Actual (R²={r2_original:.4f})', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# === 圖 3: 客戶端參與分佈 ===
ax3 = fig.add_subplot(gs[0, 2])
from collections import Counter
client_participation = Counter([c for round_clients in history['selected_clients'] for c in round_clients])
client_names_short = [name.replace('Consumer_', 'C') for name in client_participation.keys()]
ax3.bar(range(len(client_participation)), list(client_participation.values()), 
        color='skyblue', edgecolor='black', linewidth=1.5)
ax3.set_xlabel('Client', fontsize=12)
ax3.set_ylabel('Participation Count', fontsize=12)
ax3.set_title('Client Participation Distribution', fontsize=14, fontweight='bold')
ax3.set_xticks(range(len(client_participation)))
ax3.set_xticklabels(client_names_short, rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

# === 圖 4: 預測與實際折線圖（完整序列，原始尺度）===
ax4 = fig.add_subplot(gs[1, :])
sample_size = min(500, len(targets_original))  # 顯示前 500 個樣本
time_steps = np.arange(sample_size)
ax4.plot(time_steps, targets_original[:sample_size], 
         label='Actual Power Demand', color='blue', linewidth=2, alpha=0.8)
ax4.plot(time_steps, predictions_original[:sample_size], 
         label='Predicted Power Demand', color='red', linewidth=1.5, alpha=0.7, linestyle='--')
ax4.fill_between(time_steps, 
                  targets_original[:sample_size], 
                  predictions_original[:sample_size], 
                  alpha=0.2, color='gray', label='Prediction Error')
ax4.set_xlabel('Time Step', fontsize=12)
ax4.set_ylabel('Power Demand (原始值)', fontsize=12)
ax4.set_title(f'Time Series: Prediction vs Actual (前 {sample_size} 個樣本)', fontsize=14, fontweight='bold')
ax4.legend(fontsize=10, loc='upper right')
ax4.grid(True, alpha=0.3)

# === 圖 5: 預測與實際折線圖（局部放大）===
ax5 = fig.add_subplot(gs[2, 0])
zoom_size = min(100, len(targets_original))
time_steps_zoom = np.arange(zoom_size)
ax5.plot(time_steps_zoom, targets_original[:zoom_size], 
         label='Actual', color='blue', linewidth=3, marker='o', markersize=4)
ax5.plot(time_steps_zoom, predictions_original[:zoom_size], 
         label='Predicted', color='red', linewidth=2, marker='s', markersize=3, linestyle='--')
ax5.set_xlabel('Time Step', fontsize=12)
ax5.set_ylabel('Power Demand', fontsize=12)
ax5.set_title(f'局部放大 (前 {zoom_size} 個樣本)', fontsize=14, fontweight='bold')
ax5.legend(fontsize=10)
ax5.grid(True, alpha=0.3)

# === 圖 6: 誤差分佈（原始尺度）===
ax6 = fig.add_subplot(gs[2, 1])
errors_original = predictions_original - targets_original
ax6.hist(errors_original, bins=50, alpha=0.7, color='green', edgecolor='black', linewidth=1.5)
ax6.axvline(x=0, color='red', linestyle='--', linewidth=3, label='Zero Error')
ax6.axvline(x=np.mean(errors_original), color='orange', linestyle='--', linewidth=2, label=f'Mean Error: {np.mean(errors_original):.2f}')
ax6.set_xlabel('Prediction Error (原始值)', fontsize=12)
ax6.set_ylabel('Frequency', fontsize=12)
ax6.set_title(f'Error Distribution (MAE={mae_original:.2f})', fontsize=14, fontweight='bold')
ax6.legend(fontsize=10)
ax6.grid(True, alpha=0.3, axis='y')

# === 圖 7: 誤差百分比分佈 ===
ax7 = fig.add_subplot(gs[2, 2])
percentage_errors = (errors_original / targets_original) * 100
percentage_errors = percentage_errors[~np.isinf(percentage_errors)]  # 移除無窮大
ax7.hist(percentage_errors, bins=50, alpha=0.7, color='purple', edgecolor='black', linewidth=1.5)
ax7.axvline(x=0, color='red', linestyle='--', linewidth=3, label='Zero Error')
ax7.set_xlabel('Prediction Error (%)', fontsize=12)
ax7.set_ylabel('Frequency', fontsize=12)
ax7.set_title(f'Percentage Error Distribution (sMAPE={smape_value:.2f}%)', fontsize=14, fontweight='bold')
ax7.legend(fontsize=10)
ax7.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('federated_vfl_detailed_results.png', dpi=150, bbox_inches='tight')
print(f"  ✓ 詳細可視化圖表已保存: federated_vfl_detailed_results.png")
plt.show()

# ============================================================================
# 步驟 3: 性能統計摘要
# ============================================================================
print(f"\n【步驟 3】性能統計摘要")

print(f"\n標準化尺度（訓練時）:")
print(f"  - MSE: {mse:.6f}")
print(f"  - MAE: {mae:.6f}")
print(f"  - RMSE: {rmse:.6f}")
print(f"  - R²: {r2:.6f}")

print(f"\n原始尺度（實際應用）:")
print(f"  - MSE: {mse_original:.4f}")
print(f"  - MAE: {mae_original:.4f} (平均絕對誤差)")
print(f"  - RMSE: {rmse_original:.4f} (均方根誤差)")
print(f"  - R²: {r2_original:.4f} (決定係數)")
print(f"  - sMAPE: {smape_value:.2f}% (對稱平均絕對百分比誤差)")

print(f"\n誤差統計:")
print(f"  - 平均誤差: {np.mean(errors_original):.4f}")
print(f"  - 中位數誤差: {np.median(errors_original):.4f}")
print(f"  - 誤差標準差: {np.std(errors_original):.4f}")
print(f"  - 最大正誤差: {np.max(errors_original):.4f}")
print(f"  - 最大負誤差: {np.min(errors_original):.4f}")

print(f"\n聯邦學習統計:")
print(f"  - 總訓練輪數: {total_rounds}")
print(f"  - 參與客戶端總數: {len(client_names)}")
print(f"  - 每輪選擇: {num_clients} 個客戶端")
print(f"  - Weather Model 更新次數: {phase1_rounds + phase2_rounds // 5}")
print(f"  - 通訊節省比例: {(1 - (phase1_rounds + phase2_rounds // 5) / total_rounds) * 100:.1f}%")

print(f"\n{'=' * 80}")
print("VFL 聯邦學習與詳細分析完成！")
print("=" * 80)